# CNTR Stats Visualization

## Instructions

Instruction: 
1. Run the code cell 1 - 3
2. Run other cells afterwards to visualize data

In [ ]:
from IPython.core.display import HTML
def css_styling():
    styles = open("./css/style.css", "r").read()
    return HTML(styles)
css_styling()

In [ ]:
import sqlite3

players = list()
operations = list()
roles = list()

conn = sqlite3.connect('stats.sqlite')
c = conn.cursor()
for row in c.execute('SELECT * FROM Players ORDER BY Player'):
    players.append(row)
for row in c.execute('SELECT * FROM Operations ORDER BY Player'):
    operations.append(row)
for row in c.execute('SELECT * FROM Roles ORDER BY Player'):
    roles.append(row)
conn.close()

The following cell is a fix for mpld3 which relies on the NumpyEncoder. mpld3 is no longer actively developed and is using an outed version of numpys encoder.

In [ ]:
#mpld3 - numpy FIX
import json

class NumpyEncoder(json.JSONEncoder):
    """ Special json encoder for numpy types """
    def default(self, obj):
        if isinstance(obj, (numpy.int_, numpy.intc, numpy.intp, numpy.int8,
            numpy.int16, numpy.int32, numpy.int64, numpy.uint8,
            numpy.uint16,numpy.uint32, numpy.uint64)):
            return int(obj)
        elif isinstance(obj, (numpy.float_, numpy.float16, numpy.float32, 
            numpy.float64)):
            return float(obj)
        elif isinstance(obj,(numpy.ndarray,)): #### This is the fix
            return obj.tolist()
        return json.JSONEncoder.default(self, obj)
class NumpyEncoder(json.JSONEncoder):
    def default(self, obj):
        import numpy as np
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return json.JSONEncoder.default(self, obj)
from mpld3 import _display
_display.NumpyEncoder = NumpyEncoder

## Data Visualization

The following cell will display a scatter plot of the Operations/Deaths for every player. You might need to run this cell twice because it may not be formatted correctly on the first run.

In [ ]:
import matplotlib.pyplot as plt
import mpld3

fig, ax = plt.subplots(figsize=(12, 12),subplot_kw=dict(facecolor ='#EEEEEE'))
l = list() #Labels
x = list() #y-Values
y = list() #y-Values
c = list() #Colors

#Data For Plotting
for player in players:
    l.append(player[0] + "(" + str(player[1]) + ", " + str(player[2]) + ")")
    x.append(player[1])
    y.append(player[2])
    if player[2]/player[1] >= 1.10:
        c.append("r")
    elif player[2]/player[1] >= 0.90:
        c.append("k")
    else:
        c.append("g")

#Plot
#ax.plot((10, 10), (90, 90), 'k-', 100)
scatter = ax.scatter(x,
                     y,
                     c=c,
                     s=100,
                     alpha=0.3,
                     cmap=plt.cm.jet)

#Desciption
ax.set_title("Operations/Deaths - Scatter Plot of all players", size=20)
plt.xlabel('Operations')
plt.ylabel('Deaths')

#Link Tooltips
tooltip = mpld3.plugins.PointLabelTooltip(scatter, labels=l)
mpld3.plugins.connect(fig, tooltip)

mpld3.display()

The following cell will display the playerdata for a given player.

In [ ]:
from IPython.display import clear_output
from ipywidgets import widgets, Output

textField = widgets.Text()
display(textField)
out = Output()
display(out)

def handle_submit(sender):
    out.clear_output()
    ShowPlayerData(textField.value)
    
textField.on_submit(handle_submit)

In [ ]:
#from IPython.display import HTML, display
import sqlite3

def ShowPlayerData(name):
    odata = list()
    rdata = list()

    #Get Data From SQLite File
    conn = sqlite3.connect('stats.sqlite')
    c = conn.cursor()
    for row in c.execute("SELECT * FROM Players WHERE Player = '%s'" % name):
        pdata = row
    for row in c.execute("SELECT * FROM Operations WHERE Player = '%s'" % name):
        odata.append(row)
    for row in c.execute("SELECT * FROM Roles WHERE Player = '%s'" % name):
        rdata.append(row)
    conn.close()

    #Display Data
    with out:
        print("Name: " + pdata[0])
        print("Operations: " + str(pdata[1]))
        print("Deaths: " + str(pdata[2]))
        print("Death Ratio: %.2f" % (pdata[2]/pdata[1]))

        print("Roles:")
        for role in rdata:
            print("   Role: " + str(role[1])+"\t(" + str(role[2])+")")

        print("Operations:")
        for op in odata:
            print("   " + str(op[1]) + "\t" + str(op[2])+"\t(" + str(op[3])+")")


## Display Full Database

The following cell will display the stats of all players [PlayerName, Operations, Deaths]

In [ ]:
from IPython.display import HTML, display
display(HTML(
    '<table class=\"playerTable\"><tr><th>Player</th><th>Operations</th><th>Deaths</th></tr>'+
    '<tr>{}</tr></table>'.format(
        '</tr><tr>'.join(
            '<td>{}</td>'.format('</td><td>'.join(str(_) for _ in player)) for player in players)
    )
 ))

The following cell will display all operations played by each player

In [ ]:
from IPython.display import HTML, display
display(HTML(
    '<table class=\"playerTable\"><tr><th>Player</th><th>OperationName</th><th>OperationDate</th><th>OperationDeaths</th></tr>'+
    '<tr>{}</tr></table>'.format(
        '</tr><tr>'.join(
            '<td>{}</td>'.format('</td><td>'.join(str(_) for _ in operation)) for operation in operations)
    )
 ))

The following cell will display all roles played by each player

In [ ]:
from IPython.display import HTML, display
display(HTML(
    '<table class=\"playerTable\"><tr><th>Player</th><th>Role</th><th>Times Played</th></tr>'+
    '<tr>{}</tr></table>'.format(
        '</tr><tr>'.join(
            '<td>{}</td>'.format('</td><td>'.join(str(_) for _ in role)) for role in roles)
    )
 ))